In [2]:
#import

import math
import pymysql 
import requests
from urllib.parse import unquote, urlencode, quote_plus 
from datetime import datetime, date, timedelta
import json

In [3]:
river_url = "https://apis.data.go.kr/6260000/BusanRvrwtLevelInfoService/getRvrwtLevelInfo"

serviceKey = "wGokgRxD1t3z5G4u7MsWumpoCeiWO8JM6yZ87rX1ELTO9nMSUuMOQjHj70rAzuopgyB1iLdKX0S9WK0RLs88bQ==" # 공공데이터 포털에서 생성된 본인의 서비스 키를 복사 / 붙여넣기

def get_river(river_url, serviceKey):
    result = []

    serviceKeyDecoded = unquote(serviceKey, 'UTF-8') 

    queryParams = '?' + urlencode({quote_plus('serviceKey') : serviceKeyDecoded, quote_plus('pageNo') : 1,
                                        quote_plus('resultType') : 'json', quote_plus('numOfRows') : '20'}) #페이지로 안나누고 한번에 받아오기 위해 numOfRows=60으로 설정해주었다

    res = requests.get(river_url + queryParams, verify=False)
    datas = res.json()['getRvrwtLevelInfo']['body']['items']['item']

    for data in datas:
        if (data['siteName'] == '연안교' or
            data['siteName'] == '원동교' or
            data['siteName'] == '온천천 하류' or
            data['siteName'] == '중앙여고' or
            data['siteName'] == '온천장역 북측' ):
            result.append(data)

    return result

get_river(river_url, serviceKey)

c:\DEV\Langs\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'siteCode': '00-210-0002',
  'siteName': '연안교',
  'waterLevel': '1.35',
  'dayLevelMax': '1.63',
  'obsrTime': '2023-08-07 10:04',
  'alertLevel1': '1.5',
  'alertLevel1Nm': '둔치',
  'alertLevel2': '4.09',
  'alertLevel2Nm': '주의',
  'alertLevel3': '4.69',
  'alertLevel3Nm': '경계',
  'alertLevel4': '5.2',
  'alertLevel4Nm': '위험',
  'sttus': '00',
  'sttusNm': '정상'},
 {'siteCode': '00-210-0003',
  'siteName': '원동교',
  'waterLevel': '1.26',
  'dayLevelMax': '1.54',
  'obsrTime': '2023-08-07 10:04',
  'alertLevel1': '1.5',
  'alertLevel1Nm': '둔치',
  'alertLevel2': '3.67',
  'alertLevel2Nm': '주의',
  'alertLevel3': '4.42',
  'alertLevel3Nm': '경계',
  'alertLevel4': '5.04',
  'alertLevel4Nm': '위험',
  'sttus': '00',
  'sttusNm': '정상'},
 {'siteCode': '00-210-0004',
  'siteName': '온천천 하류',
  'waterLevel': '2.09',
  'dayLevelMax': '2.36',
  'obsrTime': '2023-08-07 10:04',
  'alertLevel1': '0.0',
  'alertLevel1Nm': '둔치',
  'alertLevel2': '4.01',
  'alertLevel2Nm': '주의',
  'alertLevel3': '4.43',
  '

In [33]:
def connect_to_DB():
    conn = pymysql.connect(
        host = '210.119.12.112',
        port = 10000,
        user = 'pi',
        password = '12345',
        database = 'team1_iot'
    )
    return conn

for data in result:
    try:
        conn = connect_to_DB()
        cur = conn.cursor()

        total_qry = f"""INSERT INTO riverflow
                        (siteName, waterLevel, dayLevelMax, obsrTime, alertLevel1, alertLevel1Nm, alertLevel2, alertLevel2Nm, 
                        alertLevel3, alertLevel3Nm, alertLevel4, alertLevel4Nm, sttus, sttusNm) VALUES
                            ('{data['siteName']}','{data['waterLevel']}', '{data['dayLevelMax']}', '{data['obsrTime']}','{data['alertLevel1']}', '{data['alertLevel1Nm']}',
                            '{data['alertLevel2']}', '{data['alertLevel2Nm']}','{data['alertLevel3']}', '{data['alertLevel3Nm']}','{data['alertLevel4']}', '{data['alertLevel4Nm']}',
                            '{data['sttus']}', '{data['sttusNm']}')"""
        
        cur.execute(total_qry)

        conn.commit()
        conn.close()
    except Exception as e:
        print(f'{e}')

In [3]:
url_predict= "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst" # 초단기 예보 (저장용)

# 초단기 예보 강물 데이터 API를 받아오는 소스 작성 json -> 원하는 값만 뽑아서 data랑  강수예측 % 랑 같이 DB에 업데이트 하는 코드 
 
serviceKey= "wGokgRxD1t3z5G4u7MsWumpoCeiWO8JM6yZ87rX1ELTO9nMSUuMOQjHj70rAzuopgyB1iLdKX0S9WK0RLs88bQ==" # 공공데이터 포털에서 생성된 본인의 서비스 키를 복사 / 붙여넣기

serviceKeyDecoded = unquote(serviceKey, 'UTF-8') # 공공데이터 포털에서 제공하는 서비스키는 이미 인코딩된 상태이므로, 디코딩하여 사용해야 함 -> 초단기 실황(예보도 동일)


In [4]:
def base_Time():
    now = datetime.now()
    base_time = ""

    if  now.minute > 0 and now.minute < 30 :
        base_time = f"{now.hour-1:02d}30"
    
    elif now.minute > 30:
         base_time = f"{now.hour-1:02d}00"

    return base_time

def base_Date():
    today = datetime.today()
    y = today- timedelta(days=1)
    base_date = today.strftime("%Y%m%d")

    return base_date

In [5]:
base_time = base_Time()
#'1130' -> 시간 조금 손봐야함  
base_date = base_Date()
#'20230803' -> 시간 조금 손봐야함 

queryParams = '?' + urlencode({quote_plus('serviceKey') : serviceKeyDecoded, quote_plus('base_date') : base_date, quote_plus('pageNo') : 1,
                                      quote_plus('base_time') : base_time, quote_plus('nx') : 98, quote_plus('ny') : 76,
                                      quote_plus('dataType') : 'json', quote_plus('numOfRows') : '60'}) #페이지로 안나누고 한번에 받아오기 위해 numOfRows=60으로 설정해주었다

res = requests.get(url_predict+queryParams, verify=False)
data = res.json().get('response').get('body').get('items')

data

c:\DEV\Langs\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'item': [{'baseDate': '20230803',
   'baseTime': '1630',
   'category': 'LGT',
   'fcstDate': '20230803',
   'fcstTime': '1700',
   'fcstValue': '0',
   'nx': 98,
   'ny': 76},
  {'baseDate': '20230803',
   'baseTime': '1630',
   'category': 'LGT',
   'fcstDate': '20230803',
   'fcstTime': '1800',
   'fcstValue': '0',
   'nx': 98,
   'ny': 76},
  {'baseDate': '20230803',
   'baseTime': '1630',
   'category': 'LGT',
   'fcstDate': '20230803',
   'fcstTime': '1900',
   'fcstValue': '0',
   'nx': 98,
   'ny': 76},
  {'baseDate': '20230803',
   'baseTime': '1630',
   'category': 'LGT',
   'fcstDate': '20230803',
   'fcstTime': '2000',
   'fcstValue': '0',
   'nx': 98,
   'ny': 76},
  {'baseDate': '20230803',
   'baseTime': '1630',
   'category': 'LGT',
   'fcstDate': '20230803',
   'fcstTime': '2100',
   'fcstValue': '0',
   'nx': 98,
   'ny': 76},
  {'baseDate': '20230803',
   'baseTime': '1630',
   'category': 'LGT',
   'fcstDate': '20230803',
   'fcstTime': '2200',
   'fcstValue': '0',

In [6]:
weather_data = dict()

for item in data['item']:
    Date = item['fcstDate']
    Time = item['fcstTime']
    if item['fcstDate'] not in weather_data:
        weather_data[Date] = dict()
    if item['fcstTime'] not in weather_data[Date]:
        weather_data[Date][Time] = dict()

    if item['category'] == "T1H":
        weather_data[Date][Time]['T1H'] = item['fcstValue']
    elif item['category'] == "RN1":
        weather_data[Date][Time]['RN1'] = item['fcstValue']
    elif item['category'] == "SKY":
        weather_data[Date][Time]["SKY"] = item['fcstValue']
    elif item['category'] == "UUU":
        weather_data[Date][Time]['UUU'] = item['fcstValue']
    elif item['category'] == "VVV":
        weather_data[Date][Time]['VVV'] = item['fcstValue']
    elif item['category'] == "REH":
        weather_data[Date][Time]['REH'] = item['fcstValue']
    elif item['category'] == "PTY":
        weather_data[Date][Time]['PTY'] = item['fcstValue']
    elif item['category'] == "LGT":
        weather_data[Date][Time]['LGT'] = item['fcstValue']
    elif item['category'] == "VEC":
        weather_data[Date][Time]['VEC'] = item['fcstValue']
    elif item['category'] == 'WSD':
        weather_data[Date][Time]['WSD'] = item['fcstValue']

weather_data

{'20230803': {'1700': {'LGT': '0',
   'PTY': '0',
   'RN1': '강수없음',
   'SKY': '1',
   'T1H': '33',
   'REH': '55',
   'UUU': '-0.1',
   'VVV': '1.9',
   'VEC': '173',
   'WSD': '2'},
  '1800': {'LGT': '0',
   'PTY': '0',
   'RN1': '강수없음',
   'SKY': '1',
   'T1H': '32',
   'REH': '65',
   'UUU': '-0.1',
   'VVV': '1.5',
   'VEC': '174',
   'WSD': '1'},
  '1900': {'LGT': '0',
   'PTY': '0',
   'RN1': '강수없음',
   'SKY': '3',
   'T1H': '31',
   'REH': '65',
   'UUU': '-0.4',
   'VVV': '0.5',
   'VEC': '137',
   'WSD': '1'},
  '2000': {'LGT': '0',
   'PTY': '0',
   'RN1': '강수없음',
   'SKY': '4',
   'T1H': '29',
   'REH': '70',
   'UUU': '-0.8',
   'VVV': '0.1',
   'VEC': '99',
   'WSD': '1'},
  '2100': {'LGT': '0',
   'PTY': '0',
   'RN1': '강수없음',
   'SKY': '3',
   'T1H': '29',
   'REH': '80',
   'UUU': '-1.1',
   'VVV': '0.3',
   'VEC': '103',
   'WSD': '1'},
  '2200': {'LGT': '0',
   'PTY': '0',
   'RN1': '강수없음',
   'SKY': '3',
   'T1H': '28',
   'REH': '85',
   'UUU': '-0.8',
   'VVV': '0.

In [7]:
conn = pymysql.connect(host='localhost', user='root', password='12345', db='team1_iot', charset='utf8')
cur = conn.cursor()

def connect_to_DB():
    conn = pymysql.connect(
        host = 'localhost',
        user = 'root',
        password = '12345',
        database = 'team1_iot'
    )
    return conn

In [9]:
def UpdateDB():
    try:
        for day in weather_data:
            for time in weather_data[day]:
                conn = pymysql.connect(host = 'localhost',
                                    user = 'root',
                                    password = '12345',
                                    database = 'team1_iot')
                cur = conn.cursor()

                query = '''UPDATE predict_w SET 
                                T1H = %s, RN1 = %s, SKY = %s,
                                UUU = %s, VVV = %s, REH = %s,
                                PTY = %s, LGT = %s, VEC = %s, 
                                WSD = %s  WHERE fcstDate = %s AND fcstTime = %s '''
                
                tmp_day = day[0:4] + '-' + day[4:6] + '-' + day[6:]
                tmp_time = time[0:2] + ':' + time[2:]
                cur.execute(query, (weather_data[day][time]['T1H'],weather_data[day][time]['RN1'],weather_data[day][time]['SKY'],weather_data[day][time]['UUU'],
                                    weather_data[day][time]['VVV'],weather_data[day][time]['REH'],weather_data[day][time]['PTY'],weather_data[day][time]['LGT'],
                                    weather_data[day][time]['VEC'],weather_data[day][time]['WSD'], tmp_day, tmp_time))
                conn.commit()
        conn.close()
        print('수정 완료')
    except Exception as e:
        print(f'{e}')

UpdateDB()

수정 완료


In [87]:
def InsertDB():
    try:
        for day in weather_data:
            for time in weather_data[day]:
                conn = pymysql.connect(host = 'localhost',
                                    user = 'root',
                                    password = '12345',
                                    database = 'team1_iot')
                cur = conn.cursor()
                tmp_day = day[0:4] + '-' + day[4:6] + '-' + day[6:]
                tmp_time = time[0:2] + ':' + time[2:]

                total_qry = f"""INSERT INTO predict_w 
                                (fcstDate, fcstTime, T1H, RN1, SKY, UUU, VVV, REH, PTY, LGT, VEC, WSD) VALUES
                                 ('{tmp_day}', '{tmp_time}', {weather_data[day][time]['T1H']}, 
                                  '{weather_data[day][time]['RN1']}', '{weather_data[day][time]['SKY']}', {weather_data[day][time]['UUU']}, 
                                  {weather_data[day][time]['VVV']}, {weather_data[day][time]['REH']}, '{weather_data[day][time]['PTY']}',
                                  {weather_data[day][time]['LGT']}, {weather_data[day][time]['VEC']}, {weather_data[day][time]['WSD']})"""
                
                cur.execute(total_qry)
                # cur.execute(query, (tmp_day, tmp_time, weather_data[day][time]['T1H'],weather_data[day][time]['RN1'],weather_data[day][time]['SKY'],weather_data[day][time]['UUU'],
                #                     weather_data[day][time]['VVV'],weather_data[day][time]['REH'],weather_data[day][time]['PTY'],weather_data[day][time]['LGT'],
                #                     weather_data[day][time]['VEC'],weather_data[day][time]['WSD']))
                conn.commit()
        conn.close()
        print('입력 완료')
    except Exception as e:
        print(f'{e}')

InsertDB()

입력 완료


In [10]:
if __name__ == "__main__":
    conn = pymysql.connect(host = 'localhost',
                           user = 'root',
                           password = '12345',
                           database = 'team1_iot')
    cur = conn.cursor()
    check_query = "SELECT EXISTS (SELECT idx FROM predict_w WHERE idx = 1 LIMIT 1) AS SUCCESS;"
    cur.execute(check_query)
    result = cur.fetchone()[0]

    if result == 0:
        InsertDB()
    else:
        UpdateDB()
    
    cur.close()
    conn.close()

수정 완료


DB 삽입 / 업데이트 완료 

-> 나중에 py로 옮겨야함 + 주석 

-> 침수 예측 확률값을 DB 삽입 (메인문 안에서 같이 실행시키기)